* Oskar Szudzik 148245
* Krystian Moras 148243

Our first step was to download articles from Wikipedia and save them in .csv file. You can find more code of the preprocessing in pipeline.py

Now, we need to load this data

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from scipy.spatial.distance import cosine

In [2]:
data = pd.read_csv('wikipedia.csv', header=0)
data.head()

,title,summary,links,categories,references
0,Psilocybe yungensis,psilocybe yungensis is a species of psychedeli...,Agaricales|Agaricomycetes|Alexander H. Smith|A...,Articles with 'species' microformats|Articles ...,http://www.fungimag.com/summer-2011-articles/F...
1,"Mian Rud, South Khorasan","mian rud (persian: ميان رود, also romanized as...","Abbasabad, Doreh|Abbasabad, Momenabad|Administ...",All stub articles|Articles containing Persian-...,http://geonames.nga.mil/namesgaz/|http://geoha...
2,Coat of arms of Hesse,the coat of arms of the german state of hesse...,Armiger|Blazon|Coat of arms|Coat of arms of An...,All stub articles|Articles with short descript...,https://www.hessen.de/fuer-besucher/70-jahre-h...
3,Helsinki Police Department,the helsinki police department (hpd) (finnish:...,2018 Russia–United States summit|8th World Fes...,Government agencies established in 1826|Law en...,http://tass.com/politics/1013215/amp|https://b...
4,Broadway Stages,"broadway stages, ltd. is one of new york’s ful...","Annadale, Staten Island|Arden Heights, Staten ...",1983 establishments in New York City|American ...,http://Broadway-Stages.com/|http://www.broadwa...


We will perform TF-IDF for summary of articles. To later propose similar articles to ones liked by a client.

In [3]:
article = data.iloc[0]
article.summary

'psilocybe yungensis is a species of psychedelic mushroom in the family hymenogastraceae. in north america, it is found in northeast, central and southeastern mexico. in south america, it has been recorded from bolivia, colombia, and ecuador. it is also known from the caribbean island martinique, and china. the mushroom grows in clusters or groups on rotting wood. the fruit bodies have conical to bell-shaped reddish- to orangish-brown caps that are up to 2.5 cm (1.0 in) in diameter, set atop slender stems 3 to 5 cm (1.2 to 2.0 in) long. the mushrooms stain blue when bruised, indicative of the presence of the compound psilocybin. psilocybe yungensis is used by mazatec indians in the mexican state of oaxaca for entheogenic purposes.'

In [4]:
articles = np.array(data.title)

summaries = np.array(data.summary)
summaries[:2]

array(['psilocybe yungensis is a species of psychedelic mushroom in the family hymenogastraceae. in north america, it is found in northeast, central and southeastern mexico. in south america, it has been recorded from bolivia, colombia, and ecuador. it is also known from the caribbean island martinique, and china. the mushroom grows in clusters or groups on rotting wood. the fruit bodies have conical to bell-shaped reddish- to orangish-brown caps that are up to 2.5 cm (1.0 in) in diameter, set atop slender stems 3 to 5 cm (1.2 to 2.0 in) long. the mushrooms stain blue when bruised, indicative of the presence of the compound psilocybin. psilocybe yungensis is used by mazatec indians in the mexican state of oaxaca for entheogenic purposes.',
       'mian rud (persian: ميان رود, also romanized as mīān rūd, meyan rūd, and miyan rood; also known as kalāteh miyānrūd and kalāteh-ye mīān rūd) is a village in naharjan rural district, mud district, sarbisheh county, south khorasan province, iran

In [5]:
tfidf=TfidfVectorizer(use_idf=True, smooth_idf=False) 
 
dfTFIDF = pd.DataFrame(tfidf.fit_transform(summaries).toarray(), index=articles, columns=tfidf.get_feature_names_out())
dfTFIDF.head()

,000,048,10,100,104,11,11th,12,122,1245,...,žigajevs,γεώργιος,ματσαδές,лука,اباد,الوجه,رحيم,رود,ميان,李莹
Psilocybe yungensis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
"Mian Rud, South Khorasan",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.149632,0.149632,0.0
Coat of arms of Hesse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
Helsinki Police Department,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
Broadway Stages,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


In [44]:
query = "Helsinki Police Department"
mask = data['title'] == query
pos = np.flatnonzero(mask)
query_summary = data.iloc[3].summary

query = tfidf.transform([query_summary]).toarray()[0] 
1-dfTFIDF.apply(lambda x: cosine(x, query), axis=1).sort_values()[:5]

Helsinki Police Department    1.000000
William Stephen Devery        0.243653
American Country Awards       0.187301
Al Wajh                       0.170382
Oral Çalışlar                 0.153468
dtype: float64

Okay, but we would like to apply it to a list of liked articles.

In [50]:
liked_articles = ["Helsinki Police Department", "William Stephen Devery"]

scores = {}
for l_article in liked_articles:
    mask = data['title'] == l_article
    pos = np.flatnonzero(mask)
    query_summary = data.iloc[pos[0]].summary

    query = tfidf.transform([query_summary]).toarray()[0] 
    score_board = 1 - dfTFIDF.apply(lambda x: cosine(x, query), axis=1)
    for article in score_board.keys():
        if article in scores.keys():
            scores[article] += score_board[article] / len(liked_articles)
        else:
            scores[article] = score_board[article] / len(liked_articles)

sorted_scores = {k: v for k, v in sorted(score_board.items(), key=lambda item: -item[1])}
for l_article in liked_articles:
    sorted_scores.pop(l_article)
sorted_scores

{'Premier League Riders Championship': 0.11302723506333545,
 'Coventry park and ride': 0.10927613278684178,
 '2011–12 TFF Second League': 0.1000549982196215,
 'Oral Çalışlar': 0.0948534816098211,
 'Al Wajh': 0.09321985666500554,
 'Imperial Federation': 0.09253539882605322,
 'Colony House (Keene, New Hampshire)': 0.08191057645710931,
 'Nanne Zwiep': 0.08118278938818158,
 'Fiat Materfer 7131': 0.0801357884906495,
 'American Country Awards': 0.07882782690125212,
 'Broadway Stages': 0.07619591112996194,
 'Soul Food (soundtrack)': 0.07284127016878306,
 "Breeders' Cup Distaff": 0.06971602022934997,
 'Coat of arms of Hesse': 0.06834832952998937,
 'New Inn, Carmarthenshire': 0.06720504430532148,
 'Leonard Arthur': 0.06560511115084555,
 'Flipz': 0.06540895991810769,
 'C. R. Crews': 0.0648517663341267,
 '2022 Sprecher 150': 0.06455015481668946,
 'Rincão, Cape Verde': 0.06405298851577579,
 'Psilocybe yungensis': 0.05939712004563091,
 'Ark Walworth Academy': 0.05089316261737031,
 'Puerto Rico nati